# Correlation Between Semantic Similarity and Wikipedia Distance
### CMSC 320, Spring 2022
#### Ben Landrum, Gaetan Almela, and Nav Bindra



The Wikipedia link graph is a graph where vertices represent Wikipedia articles and directed edges from vertex $A$ to vertex $B$ represent a link to $B$ in the text of $A$. We define Wikipedia distance between pages $A$ and $B$ to be the minimum number of links required to get from $A$ to $B$ in the Wikipedia link graph, where the first such link is on page $A$, the second is on the page the first link points to, and so on, until a link to $B$ is found.

Word2Vec is a popular tool for computing word embeddings, which use a vector space representation of words to place them in an abstract space where similar vectors indicate that words appear in similar contexts. These vectors can then be compared to find a metric of semantic similarity between words. 

Because pages on Wikipedia contain links which are relevant to the topic the page is covering, similar topics are often closely if not directly linked. Therefore, it stands to reason that if semantic similarity is a valid metric of similarity between topics, a higher semantic similarity between two topics should correspond to a lower Wikipedia distance between said topics. We are looking to test the hypothesis that there exists a negative correlation between Wikipedia distance and Word2Vec semantic similarity. Our null hypothesis is that there does not exist a statistically significant negative correlation between Wikipedia distance and Word2Vec semantic similarity.

## Computing Semantic Similarity with Word2Vec

__This is not hard. If we wanted to pad it we could compute our own embedding, which would probably make it seem a little less trivial and would honestly be really easy, just involves a lot of text being loaded.__

## Computing Wikipedia Distance

### Top $n$ Wikipedia Articles

English Wikipedia has over 6 million articles. A graph describing the links between all of them would be prohibitively large, so we will only consider the top $n$ articles for some large $n$. For $n > 1,000$ this is not something that can be found online, so we must compute it ourselves.

Wikipedia provides data dumps describing page views in the form of files tallying how many views each page received in a given hour. These files are available at [https://dumps.wikimedia.org/other/pageviews/](https://dumps.wikimedia.org/other/pageviews/), and date back to May 1st, 2015. The most popular pages on Wikipedia at any given time are generally determined by current events, so to factor out this bias and get a representative sample of the most popular articles we want to sample the largest possible time period, in this case May 1st, 2015 to the present day.

In [1]:
import pandas as pd
import os
import requests
from faker import Faker
import datetime
import warnings

ENGLISH_WIKIS = ['en', 'en.m'] # desktop and mobile versions of english wikipedia
PAGE_TYPE_REGEX = '^((User)|(Talk)|(Wikipedia)|(Special)|(Portal)|(File)):' # regex for filtering non-article pages
N = 10 ** 4 # number of pages to return
OUTPUT_FILE = 'ten_k_most_viewed_pages.csv' # name of output file
DATA_DIR = 'Data' # directory where the data is stored
RANDOM_FILES = 6 # Number of random files to use

# creating a Faker object which will later be used to generate random datetimes
fake = Faker()

# suppressing a warning caused by use of match groups in a regex 
warnings.filterwarnings('ignore', message="This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.")

In [2]:
# generate a random datetime for which there exists a pageview file
date = fake.date_time_between(start_date=datetime.datetime(year=2015, month=5, day=1, hour=1), end_date='now')

# get the url of the corresponding pageview file
url = f"https://dumps.wikimedia.org/other/pageviews/{date.year}/{date.year}-{date.month:02d}/pageviews-{date.strftime('%Y%m%d')}-{date.hour:02d}0000.gz"

# download and save the file
seed_file = DATA_DIR + "/" + f"pageviews-{date.strftime('%Y%m%d')}-{date.hour:02d}0000.gz"
r = requests.get(url)
with open(seed_file, 'wb') as f:
    f.write(r.content)

# importing first file to accumulate the others from
all_views = pd.read_csv(seed_file, sep=" ", header=None, names=["project", "page", "views", "null"], usecols=["project", "page", "views"])

# filtering to only pages from english wikipedia, both desktop and mobile
en_views = all_views[all_views['project'].isin(ENGLISH_WIKIS)]
# filtering user, talk and other non-article pages
views = en_views[~(en_views['page'].str.contains(PAGE_TYPE_REGEX, na=False))]

# computing views for each page 
views = views.groupby('page').sum()

Now we get the random files and add their views to the seed file, which becomes increasingly expensive as `all_views` grows.

In [3]:
start_time = datetime.datetime.now() # for timing execution
for i in range(RANDOM_FILES - 1):
    # generate a random datetime for which there exists a pageview file
    date = fake.date_time_between(start_date=datetime.datetime(year=2015, month=5, day=1, hour=1), end_date='now')

    # get the url of the corresponding pageview file
    url = f"https://dumps.wikimedia.org/other/pageviews/{date.year}/{date.year}-{date.month:02d}/pageviews-{date.strftime('%Y%m%d')}-{date.hour:02d}0000.gz"

    # download and save the file
    r = requests.get(url)
    with open(DATA_DIR + "/" + f"pageviews-{date.strftime('%Y%m%d')}-{date.hour:02d}0000.gz", 'wb') as f:
        f.write(r.content)
    print(f"{date.strftime('%Y/%m/%d:%H')}")
    
    # read the file (pandas can handle gzip directly) and perform same operations as above
    try:
        df = pd.read_csv(DATA_DIR + "/" + f"pageviews-{date.strftime('%Y%m%d')}-{date.hour:02d}0000.gz", compression='gzip', sep=" ", header=None, names=["project", "page", "views", "null"], usecols=["project", "page", "views"])
    except: # handling error caused by pandas' "C" CSV engine
        print(f"{date.strftime('%Y/%m/%d:%H')} failed" )
        os.remove(DATA_DIR + "/" + f"pageviews-{date.strftime('%Y%m%d')}-{date.hour:02d}0000.gz")
        continue

    df = df[df['project'].isin(ENGLISH_WIKIS)]
    df = df[~(df['page'].str.contains(PAGE_TYPE_REGEX, na=False))]
    df = df.groupby('page').sum()
    # merging with previous dataframe
    views = views.add(df, fill_value=0)

    # print status report
    print(f"[{i + 1}]\t{date.strftime('%Y/%m/%d:%H')}\t{len(views):,} pages\t[{datetime.datetime.now() - start_time} elapsed, {(i * 100) / RANDOM_FILES:.2f}% complete, {(datetime.timedelta(seconds=((datetime.datetime.now() - start_time).total_seconds() / (i + 1)) * (RANDOM_FILES - i - 1)))} remaining]")

2017/04/07:13
[1]	2017/04/07:13	2,448,411 pages	[0:00:22.516213 elapsed, 0.00% complete, 0:00:22.516213 remaining]


Now we save the pages and their views to a csv.

In [ ]:
views.sort_values('views', ascending=False, inplace=True)

views.head(N).to_csv(OUTPUT_FILE, index=True)

In the form shown here, this code only samples data from 6 hours, but for a more complete sample a more optimized version of this code was run over 12 hours, sampling a total of 1,105 hours of data. The top 5 million articles from this dataset are in the file `raw_views_5M.csv`.

__This will be very computationally expensive for articles which are far apart or have particularly high degrees, and will require the loading of a prohibitively large database of links between Wikipedia articles. Said database should probably be on a local SQL server.__

An SQL database of links between Wikipedia articles can be downloaded from [https://dumps.wikimedia.org/enwiki/latest/](Wikipedia's data dumps). The copy of `enwiki-latest-pagelinks.sql` used here was generated on April 20th, 2022, and is reasonably current to the state of Wikipedia at that time. The database is a table of links between Wikipedia articles, with each row representing a link from one article to another.
<!-- 
In order to generate the SQLite database we're using to access the graph, we run the following on the command line:
```
sqlite3 pagelinks.db ".read enwiki-latest-pagelinks.sql"
```
This takes forever, but allows us to access the graph with SQLite. -->

## Building a Bot With Semantic Similarity

__This is virtually already done, but we can make it much better and I would love to compare different iterations a la chess bot comparison grid.__

### Using Semantic Similarity to Approximate Wikipedia Distance

__This seems pointless but makes the statistical analysis straightforward.__

## Conclusion

__We can just compute the two values for random pairs of articles until we have a super low p-value and say boom statistically significant correlation. Will have to consult stat notes to remember what test you're supposed to use for correlation. $\chi^2$?__